# 机器学习工程师纳米学位
## 监督学习
## 项目 2: 搭建一个学生干预系统

欢迎来到机器学习工程师纳米学位的第二个项目！在此文件中，有些示例代码已经提供给你，但你还需要实现更多的功能让项目成功运行。除非有明确要求，你无须修改任何已给出的代码。以**'练习'**开始的标题表示接下来的代码部分中有你必须要实现的功能。每一部分都会有详细的指导，需要实现的部分也会在注释中以**'TODO'**标出。请仔细阅读所有的提示！

除了实现代码外，你还**必须**回答一些与项目和你的实现有关的问题。每一个需要你回答的问题都会以**'问题 X'**为标题。请仔细阅读每个问题，并且在问题后的**'回答'**文字框中写出完整的答案。我们将根据你对问题的回答和撰写代码所实现的功能来对你提交的项目进行评分。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

### 问题 1 - 分类 vs. 回归
*在这个项目中你的任务是找出那些如果不给予帮助，最终可能无法毕业的学生。你觉得这个问题是哪种类型的监督学习问题，是分类问题还是回归问题？为什么？*

**答案: **此问题属于分类问题，因为分类问题的显著特征就是二分，而我们需要对学生做出判断的结果是“需要给予帮助”、“不需要给予帮助”这两类，所以是分类问题。

## 分析数据
运行下面区域的代码以载入学生数据集，以及一些此项目所需的Python库。注意数据集的最后一列`'passed'`是我们的预测的目标（表示学生是毕业了还是没有毕业），其他的列是每个学生的属性。

In [1]:
# 载入所需要的库
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# 载入学生数据集
student_data = pd.read_csv("student-data.csv")
print ("Student data read successfully!")

Student data read successfully!


### 练习: 分析数据
我们首先通过调查数据，以确定有多少学生的信息，并了解这些学生的毕业率。在下面的代码单元中，你需要完成如下的运算：
- 学生的总数， `n_students`。
- 每个学生的特征总数， `n_features`。
- 毕业的学生的数量， `n_passed`。
- 未毕业的学生的数量， `n_failed`。
- 班级的毕业率， `grad_rate`， 用百分数表示(%)。


In [2]:
# TODO： 计算学生的数量
n_students = student_data.shape[0]

# TODO： 计算特征数量
n_features = student_data.shape[1]-1

# TODO： 计算通过的学生数
n_passed = student_data[student_data.passed == 'yes'].shape[0]

# TODO： 计算未通过的学生数
n_failed = n_students - n_passed

# TODO： 计算通过率
grad_rate = n_passed/n_students*100

# 输出结果
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## 数据准备
在这个部分中，我们将要为建模、训练和测试准备数据
### 识别特征和目标列
你获取的数据中通常都会包含一些非数字的特征，这会导致一些问题，因为大多数的机器学习算法都会期望输入数字特征进行计算。

运行下面的代码单元将学生数据分成特征和目标列看一看他们中是否有非数字特征。

In [3]:
# 提取特征列
feature_cols = list(student_data.columns[:-1])

# 提取目标列 ‘passed’
target_col = student_data.columns[-1] 

# 显示列的列表
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# 将数据分割成特征数据和目标数据（即X_all 和 y_all）
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# 通过打印前5行显示特征信息
print ("\nFeature values:")
print (X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### 预处理特征列

正如你所见，我们这里有几个非数值的列需要做一定的转换！它们中很多是简单的`yes`/`no`，比如`internet`。这些可以合理地转化为`1`/`0`（二元值，binary）值。

其他的列，如`Mjob`和`Fjob`，有两个以上的值，被称为_分类变量（categorical variables）_。处理这样的列的推荐方法是创建和可能值一样多的列（如：`Fjob_teacher`，`Fjob_other`，`Fjob_services`等），然后将其中一个的值设为`1`另外的设为`0`。

这些创建的列有时候叫做 _虚拟变量（dummy variables）_，我们将用[`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies)函数来完成这个转换。运行下面代码单元的代码来完成这里讨论的预处理步骤。

In [4]:
def preprocess_features(X):
    ''' 预处理学生数据，将非数字的二元特征转化成二元值（0或1），将分类的变量转换成虚拟变量
    '''
    
    # 初始化一个用于输出的DataFrame
    output = pd.DataFrame(index = X.index)

    # 查看数据的每一个特征列
    for col, col_data in X.iteritems():
        
        # 如果数据是非数字类型，将所有的yes/no替换成1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # 如果数据类型是类别的（categorical），将它转换成虚拟变量
        if col_data.dtype == object:
            # 例子: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # 收集转换后的列
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### 实现: 将数据分成训练集和测试集
现在我们已经将所有的 _分类的（categorical）_ 特征转换成数值了。下一步我们将把数据（包括特征和对应的标签数据）分割成训练集和测试集。在下面的代码单元中，你需要完成下列功能：
- 随机混洗切分数据(`X_all`, `y_all`) 为训练子集和测试子集。
  - 使用300个数据点作为训练集（约76%），使用95个数据点作为测试集（约24%）。
  - 如果可能的话，为你使用的函数设置一个`random_state`。
  - 将结果存储在`X_train`, `X_test`, `y_train`和 `y_test`中。

In [5]:
# TODO：在这里导入你可能需要使用的另外的功能
from sklearn.model_selection import train_test_split
# TODO：设置训练集的数量
num_train = 300

# TODO：设置测试集的数量
num_test = X_all.shape[0] - num_train

# TODO：把数据集混洗和分割成上面定义的训练集和测试集
X_train ,X_test,y_train,y_test = train_test_split(X_all,y_all,train_size=300,random_state=42)

# 显示分割的结果
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


## 训练和评价模型
在这个部分，你将选择3个适合这个问题并且在`scikit-learn`中已有的监督学习的模型。首先你需要说明你选择这三个模型的原因，包括这些数据集有哪些特点，每个模型的优点和缺点各是什么。然后，你需要将这些模型用不同大小的训练集（100个数据点，200个数据点，300个数据点）进行训练，并用F<sub>1</sub>的值来衡量。你需要制作三个表，每个表要显示训练集大小，训练时间，预测时间，训练集上的F<sub>1</sub>值和测试集上的F<sub>1</sub>值（每个模型一个表）。

**这是目前** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **里有的监督学习模型，你可以从中选择:**
- Gaussian Naive Bayes (GaussianNB) 朴素贝叶斯
- Decision Trees 决策树
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM) 向量模型机
- Logistic Regression 逻辑回归

### 问题 2 - 应用模型
*列出三个适合这个问题的监督学习算法模型。每一个你选择的模型：*

- 描述一个该模型在真实世界的一个应用场景。（你需要为此做点研究，并给出你的引用出处）
- 这个模型的优势是什么？他什么情况下表现最好？
- 这个模型的缺点是什么？什么条件下它表现很差？
- 根据我们当前数据集的特点，为什么这个模型适合这个问题。

**回答: **
#### 我准备选用的3个监督学习算法模型分别为：
##### 1、Decision Tree:
- 应用场景：
  - 在P0项目中对泰达尼克号的乘客进行生存分析，就是一个简单的决策树模型的应用，还有kaggle上比较好玩的[ What's Cooking?](https://www.kaggle.com/yuenyuenwha/whats-cooking/randomffff/code)。
- 模型优势：
  - 计算容易，可解释性强。
  - 比较适合处理有缺失属性的样本。决策树算法对特征相互间的交互关系不进行关联处理，也就可以避免异常值的干扰，能够处理有缺失属性的数据。
  - 能够在较短时间内对大型数据做出可行且效果良好的结果。
- 模型缺点：
  - 容易发生过拟合
  - 忽略了特征之间的相关性
  - 对各类别样本数量不一致的数据，在决策树中，信息增益的结果偏向于那些具有更多数值的特征。
- 与给定数据集切合度：
  - 本项目可以根据学生的一系列特征，按照设定条件进行逐层划分，符合决策树的算法原理。

##### 2、Logistic Regression:
- 应用场景：
  - 应用到许多的工程技术和信息过滤等方面.例如[广告点击预测](https://www.kaggle.com/datayo/avito-context-ad-clicks/logistic-regression/code)等方面。
- 模型优势：
  - 实现简单，广泛的应用于工业问题上。
  - 分类时计算量非常小，速度很快，存储资源低。
  - 便利的观测样本概率分数。
  - 可以结合L2正则化来解决多重共线性。
  - 可以使用新数据更新模型。
- 模型缺点：
  - 当特征空间很大时，逻辑回归的性能不是很好
  - 容易欠拟合，一般准确度不太高
  - 不能很好地处理大量多类特征或变量
  - 对于非线性特征，需要进行转换
- 与给定数据集切合度：
  - 本项目给定数据集为二分类问题，特征数量不大，数据量小，并且数据集具有良好的扩展性。

##### 3、SVM:
- 应用场景：
  - 目前支持向量机主要应用在模式识别领域中的[文本分类](http://blog.csdn.net/zhzhl202/article/details/8197109),人脸识别等;
- 模型优势：
  - SVM在小样本训练集上能够得到比其它算法好很多的结果。
  - 基于结构风险最小化原则，避免了overfitting，泛化能力强。
  - SVM最终决策函数只由少数支持向量决定，计算的复杂性取决于支持向量的数目，而不是样本空间的维数，这就有效避免了“维度灾难”。
- 模型缺点： 
  - SVM算法对大规模训练样本难以实施。
  SVM的空间消耗主要是存储训练样本和核矩阵，由于SVM是借助二次规划来求解支持向量，而求解二次规划将涉及m阶矩阵的计算（m为样本的个数），当m数目很大时该矩阵的存储和计算将耗费大量的机器内存和运算时间。
  - 用SVM解决多分类问题存在困难
  经典的支持向量机算法只给出了二类分类的算法，而在数据挖掘的实际应用中，一般要解决多类的分类问题。可以通过多个二类支持向量机的组合来解决。
  - 对缺失数据敏感，对参数和核函数的选择敏感
  支持向量机性能的优劣主要取决于核函数的选取,所以对于一个实际问题而言,如何根据实际的数据模型选择合适的核函数从而构造SVM算法.目前比较成熟的核函数及其参数的选择都是人为的,根据经验来选取的,带有一定的随意性.在不同的问题领域,核函数应当具有不同的形式和参数,所以在选取时候应该将领域知识引入进来,但是目前还没有好的方法来解决核函数的选取问题.
- 与给定数据集切合度：
  - 本文给定的数据集样本量较小，能够克服SVM占用内存较大的缺点；本项目数据集为二分类问题，可以发挥SVM的特点；本项目获取的数据非常干净，没有缺失值，可以避免SVM对缺失值敏感的缺陷。


### 准备
运行下面的代码单元以初始化三个帮助函数，这三个函数将能够帮你训练和测试你上面所选择的三个监督学习算法。这些函数是：
- `train_classifier` - 输入一个分类器和训练集，用数据来训练这个分类器。
- `predict_labels` - 输入一个训练好的分类器、特征以及一个目标标签，这个函数将帮你做预测并给出F<sub>1</sub>的值.
- `train_predict` - 输入一个分类器以及训练集和测试集，它可以运行`train_clasifier`和`predict_labels`.
 - 这个函数将分别输出训练集的F<sub>1</sub>值和测试集的F<sub>1</sub>值

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' 用训练集训练分类器 '''
    
    # 开始计时，训练分类器，然后停止计时
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' 用训练好的分类器做预测并输出F1值'''
    
    # 开始计时，作出预测，然后停止计时
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # 输出并返回结果
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' 用一个分类器训练和预测，并输出F1值 '''
    
    # 输出分类器名称和训练集大小
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # 训练一个分类器
    train_classifier(clf, X_train, y_train)
    
    # 输出训练和测试的预测结果
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.\n".format(predict_labels(clf, X_test, y_test)))

### 练习: 模型评价指标
借助于上面定义的函数，你现在需要导入三个你选择的监督学习模型，然后为每一个模型运行`train_predict`函数。请记住，对于每一个模型你需要在不同大小的训练集（100，200和300）上进行训练和测试。所以，你在下面应该会有9个不同的输出（每个模型都有训练集大小不同的三个输出）。在接下来的代码单元中，你将需要实现以下功能：
- 引入三个你在上面讨论过的监督式学习算法模型。
- 初始化三个模型并将它们存储在`clf_A`， `clf_B` 和 `clf_C`中。
 - 如果可能对每一个模型都设置一个`random_state`。
 - **注意:** 这里先使用每一个模型的默认参数，在接下来的部分中你将需要对某一个模型的参数进行调整。
- 创建不同大小的训练集用来训练每一个模型。
 - *不要再混洗和再分割数据！新的训练集要取自`X_train`和`y_train`.*
- 对于每一个模型要用不同大小的训练集来训练它，然后在测试集上做测试（总共需要9次训练测试）   
**注意:** 在下面的代码单元后面我们提供了三个表用来存储你的结果。

In [7]:
# TODO：从sklearn中引入三个监督学习模型
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVC

# TODO：初始化三个模型
clf_A = tree.DecisionTreeClassifier(random_state=42)
clf_B = LogisticRegression(random_state=42)
clf_C = SVC(random_state=42)

# TODO：设置训练集大小
X_train_100 = X_train.loc[:100,]
y_train_100 = y_train.loc[:100]

X_train_200 = X_train.loc[:200,]
y_train_200 = y_train.loc[:200]

X_train_300 = X_train.loc[:300,]
y_train_300 = y_train.loc[:300]

# TODO：对每一个分类器和每一个训练集大小运行'train_predict' 

DT_tp_100=train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
DT_tp_200=train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
DT_tp_300=train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

LR_tp_100=train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
LR_tp_200=train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
LR_tp_300=train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

SVM_tp_100=train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
SVM_tp_200=train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
SVM_tp_300=train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)


Training a DecisionTreeClassifier using a training set size of 185. . .
Trained model in 0.0020 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.7132.

Training a DecisionTreeClassifier using a training set size of 196. . .
Trained model in 0.0020 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7246.

Training a DecisionTreeClassifier using a training set size of 211. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7040.

Training a LogisticRegression using a training set size of 185. . .
Trained model in 0.0670 seconds
Made predictions in 0.0190 seconds.
F1 score for training set: 0.8603.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7883.

Training a LogisticRegression us

### 结果表格
编辑下面的表格看看在[Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables)中如何设计一个表格。你需要把上面的结果记录在表格中。

** 分类器 1 - Decision Tree**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0010               |       0.0000              |      1.0000            |          0.7132    |
| 200               |        0.0000          |        0.0000        |      1.0000      |       0.7246      |
| 300               |        0.0000           |       0.0000            |     1.0000         |    0.7040      |

** 分类器 2 - Logistic Regression**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0000       |        0.0000         |      0.8603        |      0.7883      |
| 200               |        0.0000          |       0.0000           |     0.8552         |     0.7971       |
| 300               |      0.0000            |       0.0000            |      0.8599        |    0.7914      |

** 分类器 3 - SVM**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0000            |          0.0000         |      0.8651         |      0.7733        |
| 200               |        0.0000          |          0.0000        |        0.8701       |       0.7815      |
| 300               |       0.0156              |       0.0000                 |      0.8709            |    0.7815      |

## 选择最佳模型
在最后这一部分中，你将从三个监督学习模型中选择一个用在学生数据上的最佳模型。然后你将在最佳模型上用全部的训练集（`X_train`和`y_train`）运行一个网格搜索算法，在这个过程中，你要至少调整一个参数以提高模型的F<sub>1</sub>值（相比于没有调参的模型的分值有所提高）。 

### 问题 3 - 选择最佳模型
*给予你上面做的实验，用一到两段话，向（学校）监事会解释你将选择哪个模型作为最佳的模型。哪个模型在现有的数据，有限的资源、开支和模型表现综合来看是最好的选择？*

**回答: **从选择的3个模型来比对，训练消耗时间最少的为Decision Tree算法，但是在训练集上表现过好，形成了overfitting，测试集中结果并不是最好的；测试结果表现最好的为LogisticRegression算法，且消耗时间，资源都为居中，模型在测试集和训练集上表现很好；SVM算法消耗资源相较其他两种算法较多，得到的结果相对处于中间位置，并且在训练集上表现最好。综合考虑，选择LR模型进行建模。

### 问题 4 - 用通俗的语言解释模型
*用一到两段话，向（学校）监事会用外行也听得懂的话来解释最终模型是如何工作的。你需要解释所选模型的主要特点。例如，这个模型是怎样被训练的，它又是如何做出预测的。避免使用高级的数学或技术术语，不要使用公式或特定的算法名词。*

**回答: **
 - 我们通过寻找学生的一些表现行为及学生周围环境来预测学生最终能否通过考试，毕竟人类是环境的产物。我们根据学生的性别、年龄、家庭距离学校距离的远近家庭成员人数，父母受教育的程度及父母的工作类型，学生上学的通勤时间，每周投入学习的时间长短，历史上考试成败的次数，家庭是否能提供额外的教育支持和付费课程，学生给你自己是否有受更高等的教育愿望，是否饮酒及旷课等表现，来做出一个能够符合学生最终能否通过考试的结果的估计，
 - 比如说，我们发现一个通过了最终考试的女学生，我们发现她具有这样一些特点：父母都受过良好教育，且家庭生活和谐，上学的地方离家近，自己得到家庭很好的教育支持，课外兴趣广泛，不旷课不酗酒；又发现一个未通过最终考试的男学生，他具有这样一些特点：来自农村，家庭生活不幸福，父母离异且俱未受过高等教育，学生自己平时不爱学习，课余经常与朋友抽烟喝酒，旷课谈恋爱，那么我们就能找出学生表现行为和家庭环境对学生最终能否通过考试的做出判断。接下来，我们就可以根据一个学生的行为表现和家庭环境来预测该生“是否能通过最终的考试”。


### 练习: 模型调参
细调选择的模型的参数。使用网格搜索（`GridSearchCV`）来至少调整模型的重要参数（至少调整一个），这个参数至少需给出并尝试3个不同的值。你要使用整个训练集来完成这个过程。在接下来的代码单元中，你需要实现以下功能：
- 导入 [`sklearn.model_selection.GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 和 [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- 创建一个对于这个模型你希望调整参数的字典。
 - 例如: `parameters = {'parameter' : [list of values]}`。
- 初始化你选择的分类器，并将其存储在`clf`中。
- 使用`make_scorer` 创建F<sub>1</sub>评分函数并将其存储在`f1_scorer`中。
 - 需正确设定参数`pos_label`的值！
- 在分类器`clf`上用`f1_scorer` 作为评价函数运行网格搜索,并将结果存储在`grid_obj`中。
- 用训练集(`X_train`, `y_train`)训练grid search object,并将结果存储在`grid_obj`中。

In [19]:
# TODO: 导入 'GridSearchCV' 和 'make_scorer'
from  sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit

# TODO：创建你希望调整的参数列表
parameters = {'C': [0.1, 0.5, 1, 10]}

# TODO：初始化分类器
clf = LogisticRegression(random_state=42)

# TODO：用'make_scorer'创建一个f1评分函数
f1_scorer = make_scorer(f1_score,pos_label='yes')

# TODO：在分类器上使用f1_scorer作为评分函数运行网格搜索

ssscv = StratifiedShuffleSplit( y_train, n_iter=10, test_size=0.1,random_state=42)
grid_obj = GridSearchCV(clf,parameters,cv = ssscv,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
# TODO：用训练集训练grid search object来寻找最佳参数
grid_obj.fit(X_train,y_train)


# Get the estimator
# 得到预测的结果
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
# 输出经过调参之后的训练集和测试集的F1值
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 0.8304.
Made predictions in 0.0005 seconds.
Tuned model has a testing F1 score of 0.7857.


### 问题 5 - 最终的 F<sub>1</sub> 值
*最终模型的训练和测试的F<sub>1</sub>值是多少？这个值相比于没有调整过参数的模型怎么样？*

**回答: **最终模型的训练F1值为0.8304，测试F1值为0.7857，相比于第2个问题中默认的LR未调整参数的值0.8599和0.7914，在训练集上的f1值出现了下降，在测试集上的f1值，同样下降了。进行模型参数调整后，F1值不升反降，模型的表现没有得到提升，也出现过训练F1值为0.8501，测试F1值为0.8029，这证明给定的默认参数是合理和准确的，自己选择的参数，由于数据划分的原因，得分在某个值附近波动。

> **注意**: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。  